In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
'''
Importation des bibliothèques nécessaires
'''
import gc
import math

from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Embedding, concatenate, Flatten
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop
from tensorflow.keras import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.losses import mean_squared_error as mse_loss
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 64
plt.rcParams['figure.figsize'] = [12, 8]
%matplotlib inline

In [ ]:
#Importation de la Dataset et afficher taille/forme

building = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")
weather_test = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv")
test = pd.read_csv("../input/ashrae-energy-prediction/test.csv")

print("Size of building data", building.shape)
print("Size of weather_train data ", weather_train.shape )
print("Size of weather_test data",weather_test.shape )
print("Size of train data" , train.shape)
print("Size of test data", test.shape)


In [ ]:
#Exploratory building_metadata.csv
building.head()

In [ ]:
#Exploratory tweather_rain.csv
weather_train.head()

In [ ]:
#Exploratory train.csv
train.head()

In [ ]:
#data processing pour (building_metadata.csv)
def process_building_data(building_df):
    
    #Ici on a le choix d'effacer les floor_count ou bien de les remplacer par mean()
    #En effet on a choisit de remplacer par la moyenne 
    
    print('Data cleaning: remplacement des valeurs nulles pour la colonne floor_count')
    building_df.floor_count.fillna(round(building_df.floor_count.mean(),0), inplace=True)
  
    print('Création d une colonne building_age')
    current_year = datetime.now().year
    building_df['building_age'] = current_year - building_df['year_built']
    
    print('Data Cleaning: Suppression de la colonne floor_count')
    building_df.drop(columns=['year_built'], inplace=True)

    print('Data cleaning: Remplacement des valeurs nulles dans la colonne building_age')
    building_df.building_age.fillna(round(building_df.building_age.mean(),0), inplace=True)

    print('Label encoding primary use')
    le = LabelEncoder()
    le_primary_use = le.fit_transform(building.primary_use)

    print('Feature engineering: Normalisation logarithmique pour la colonne square_feet')
    building_df['log_square_feet'] = np.log(building_df['square_feet'])
    building_df.drop(columns=['square_feet'], inplace=True)

    building_df['primary_use'] = le_primary_use
    #On efface maintenain : current_year, le & le_primary_use
    del current_year, le, le_primary_use
    gc.collect()

    return building_df

In [ ]:
#data processing forweather dataset
def convert_season(month):
    if (month <= 2) | (month == 12):
        return 0
    # 0 = c'est la saison d'Hiver 
    elif month <= 5:
        return 1
    # 1 = c'est le printemp
    elif month <= 8:
        return 2
    # 2 = été 
    elif month <= 11:
        return 3
    # saison automne

def convert_direction(degrees):
    if degrees <= 90:
        return 0
    # On est dans la direction  nord-est
    elif degrees <= 180:
        return 1
    # On est dans la direction sud-est
    elif degrees <= 270:
        return 2
    # direction sud-ouest
    elif degrees <= 360:
        return 3
    # direction nord-ouest

In [ ]:
#creation de l'échelle de beaufort 
#explication 
'''
L'échelle de Beaufort est une échelle de mesure empirique,
comportant 13 degrés (de 0 à 12), de la vitesse moyenne du vent sur une durée 
de dix minutes utilisée dans les milieux maritimes. 
'''
def add_beaufort_scale(df):
  beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
          (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

  for item in beaufort:
    df.loc[(df['wind_speed'] >= item[1]) & (df['wind_speed'] < item[2]), 'beaufort_scale'] = item[0]
  return df


In [ ]:
#Remplacer les données de wheather qui sont null
def replace_weather_data_nans(weather_df):
  weather_df = weather_df.set_index(['site_id','day_of_month','month'])
                        
  weather_features_with_nan = {
      'air_temperature': {'round': 1, 'fill': False},
      'dew_temperature': {'round': 1, 'fill': False},
      'wind_speed': {'round': 1, 'fill': False},
      'wind_direction': {'round': 0, 'fill': False},
      'sea_level_pressure': {'round': 1, 'fill': True},
      'precip_depth_1_hr': {'round': 0, 'fill': True},
      'cloud_coverage': {'round': 0, 'fill': True}
  }

  for feature, params in weather_features_with_nan.items():
    print('Data cleaning: Replacing NAN values for ' + feature)
    filler_df = pd.DataFrame(weather_df.groupby(['site_id','day_of_month','month'])[feature].mean().round(params['round']),
                             columns=[feature])

    if params['fill']:
      filler_df_mean = round(filler_df[feature].mean(),params['round'])
      filler_df.fillna(filler_df_mean, inplace=True)
    #créer un dataframe de air_temperatures à remplir
    temporary_df = pd.DataFrame({feature : weather_df[feature]})

    # mettre à jour les valeurs NA de air_temperatures
    temporary_df.update(filler_df, overwrite=False)

    # mise à jour dans weather dataset
    weather_df[feature] = temporary_df[feature]
  weather_df = weather_df.reset_index()
  return weather_df

In [ ]:
def process_weather_data(weather_df):
  print('Data cleaning: conversion de la colonne timestamp')
  weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

  print('Ajout de colonnes de mois, jour de semaine, jour de mois et heure')
  weather_df['month'] = weather_df['timestamp'].dt.month
  weather_df['day_of_week'] = weather_df['timestamp'].dt.dayofweek
  weather_df['day_of_month']= weather_df['timestamp'].dt.day
  weather_df['hour'] = weather_df['timestamp'].dt.hour

  print('djout d une colonne  is_weekend')
  weather_df['is_weekend'] = weather_df.day_of_week.apply(lambda x: 1 if x>=5 else 0)

  print('ajout de la colone  season feature')
  weather_df['season'] = weather_df.month.apply(convert_season)

  print('data cleaning:remplacement des valeurs manquantes')
  weather_df = replace_weather_data_nans(weather_df)

  print('ajout de la colonne de Beaufort')
  weather_df = add_beaufort_scale(weather_df)

  print('ajout de colonne wind_compass_direction ')
  weather_df['wind_direction'+'_compass'] = weather_df.wind_direction.apply(convert_direction)

  print('ajout colone wind_direction_sin ')
  weather_df['wind_direction'+'_sin'] = np.sin((2*np.pi*weather_df['wind_direction'])/360)

  print('data cleaning: suppression des la colonne wind_direction')
  weather_df.drop(columns=['wind_direction'], inplace=True)

  return weather_df


In [ ]:
#On définie  la fonction de reduction de l'utilisation de la mémoire       
def reduce_memory_usage(df):
  d_types = {'building_id': np.uint16,
            'site_id': np.uint8,
            'meter': np.uint8,
            'primary_use': np.uint8,
            'log_square_feet': np.float16,
            'floor_count': np.uint8,
            'building_age': np.uint8,
            'air_temperature': np.float16,
            'dew_temperature': np.float16,
            'wind_speed': np.float16,
            'cloud_coverage': np.uint8,
            'precip_depth_1_hr': np.int16,
            'sea_level_pressure': np.float32,
            'wind_direction_sin': np.float16,
            'wind_direction_compass': np.uint8,
            'beaufort_scale': np.uint8,
            'day_of_month': np.uint8,
            'month': np.uint8,
            'day_of_week': np.uint8,
            'hour': np.uint8,
            'is_weekend': np.uint8,
            'season': np.uint8
            }

  for feature in d_types:
    df[feature] = df[feature].astype(d_types[feature])
  return df
        


In [ ]:
def data_processing_pipeline(df, weather_df, building_df, target_variable='meter_reading'):

  print('Fixer timestamp')
  df["timestamp"] = pd.to_datetime(df["timestamp"])
  
  print('\nTraitement des données building_metadata:\n')
  building_df_copy = building_df.copy()
  building_df_copy = process_building_data(building_df_copy)

  print('\nTraitement des données weather data:\n')
  weather_df_copy = weather_df.copy()
  weather_df_copy = process_weather_data(weather_df_copy)

  print('\nFusion des  datasets:\n')
  print('Fusion avec building dataset')
  df = df.merge(building_df_copy, on='building_id', how='left')
  print('Fusion avec weather dataset')
  df = df.merge(weather_df_copy, on=['site_id', 'timestamp'], how='left')

  print('\nData cleaning: Attribuer les donnéé manquantes aprés fusion à  de lensemble')
  columns_with_nan = df.columns[df.isna().any()]
  for feature in columns_with_nan:
    df[feature] = df[feature].fillna(method='ffill')

  print('\nréduction de l utilisation de mémoire:\n')
  start_mem = df.memory_usage().sum() / 1024**2
  print('L utilisation de la mémoire de la dataframe est {:.2f} MB'.format(start_mem))
  df = reduce_memory_usage(df)
  end_mem = df.memory_usage().sum() / 1024**2
  print('utilisation de la mémoire aprés optimization is: {:.2f} MB'.format(end_mem))
  print('Donc l utilisation de la mémoire a été diminué de  {:.1f}%\n'.format(100 * (start_mem - end_mem) / start_mem))

  del weather_df_copy, building_df_copy, start_mem, end_mem
  gc.collect()
  return df

In [ ]:
from datetime import datetime
%time

train_copy = data_processing_pipeline(train, weather_train , building)


First Model creation : (Deep Neural Network)

In [ ]:
#Definisons d'abord l'erreur 
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [ ]:
def create_model(dense_layer_sizes, opt="adam", dropout=0.1, nbr_features=20, dense_nparams=256, lr=0.01):
    optimizers = {
        'adam': Adam(lr),
        'rmsprop': RMSprop(lr)
    }
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(nbr_features,))) 
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mse', optimizer=optimizers[opt], metrics=[root_mean_squared_error])
    return model


In [ ]:
model = create_model(dense_layer_sizes=[256, 128, 64, 32], dense_nparams=512, dropout=0.2)

In [ ]:
model.summary()

In [ ]:
EPOCHS = 50
BATCH_SIZE = 1024
def step_decay(epoch):
   initial_lrate = 0.01
   drop = 0.9
   lrate = initial_lrate * math.pow(drop, math.floor((epoch)))
   return lrate

best_model_file = "epochs:{epoch:03d}-val_loss:{val_loss:.3f}.h5"

mc = ModelCheckpoint(best_model_file, monitor='val_loss', mode='auto',verbose=True, save_best_only=True)
es = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True, mode='auto', restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=True, min_delta=0.001, min_lr=1e-5)
lrs = LearningRateScheduler(step_decay, verbose=True)

In [ ]:
#Train-test split
unnecessary_features = ['timestamp', 'day_of_month']

categoricals = ['building_id', 'site_id', 'meter', 'primary_use', 'floor_count',
                'cloud_coverage', 'beaufort_scale', 'wind_direction_compass',
                'day_of_week', 'hour', 'is_weekend', 'season']

numericals = ["building_age", "log_square_feet", "air_temperature", "dew_temperature",
              "precip_depth_1_hr", "sea_level_pressure", "wind_speed", "wind_direction_sin"]

target = "meter_reading"

features = categoricals + numericals


In [ ]:
# TRAIN_SIZE = 1047552
# train_sample = train.sample(TRAIN_SIZE)

X = train_copy[features]
y = train_copy[target].map(np.log1p)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
hist = model.fit(X_train.values, y_train.values, epochs=EPOCHS, batch_size=BATCH_SIZE, 
                 validation_data=(X_test.values, y_test.values), verbose=True, callbacks=[es, mc, rlr])

In [ ]:
Second method: KFold Cross validation


Second method: KFold Cross validation


In [ ]:
keras_estimator = KerasRegressor(create_model, epochs=20, batch_size=1024, verbose=2)
keras_estimator.get_params()

In [ ]:

dense_size_candidates = [[64, 32], [128, 64, 32], [256, 128, 64, 32], 
                         [256, 128, 64, 64, 32, 16], [512, 256, 128, 128, 64, 32]]
param_grid = {
    'dense_nparams': [128, 256, 384, 512, 640, 768, 896, 1024],
    'dense_layer_sizes': dense_size_candidates,
    'opt':['rmsprop', 'adam'],
    'lr': [0.001, 0.002, 0.05, 0.01],
    'dropout': [0.5, 0.3, 0.2, 0.1, 0]
}

In [ ]:
kfold_splits = 4
tuner = RandomizedSearchCV(estimator=keras_estimator,
                    verbose=True,
                    n_iter=25,
                    n_jobs=4,
                    return_train_score=True,
                    cv=kfold_splits,
                    param_distributions=param_grid)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

kfold_splits = 4
tuner = RandomizedSearchCV(estimator=keras_estimator,
                    verbose=True,
                    n_iter=25,
                    n_jobs=4,
                    return_train_score=True,
                    cv=kfold_splits,
                    param_distributions=param_grid)

In [ ]:
 es = EarlyStopping(monitor='loss', min_delta=0.01, patience=5, verbose=True, mode='auto', restore_best_weights=True)
 rlr = ReduceLROnPlateau(monitor='loss', patience=2, factor=0.75, verbose=True, min_delta=0.01, min_lr=1e-5)

 tuner_result = tuner.fit(X_train, y_train, callbacks=[es, rlr])

 # Affichage du résultats.
 print("Best: %f using %s" % (tuner_result.best_score_, tuner_result.best_params_))
 means = tuner_result.cv_results_['mean_test_score']
 stds = tuner_result.cv_results_['std_test_score']
 params = tuner_result.cv_results_['params']
 for mean, stdev, param in zip(means, stds, params):
     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
Third method LSTM 

In [ ]:
%%time
test_copy = data_processing_pipeline(test, weather_test, building)

In [ ]:
test_data = test_copy[features]
y_pred = best_model.predict(test_data.values, batch_size=1024, verbose=1)


In [ ]:
#### Data processing pipeline on test dataset


Creation OF sample_submission.csv 


In [ ]:
sample_submission['meter_reading'] = np.expm1(y_pred)

In [ ]:
export_csv = sample_submission.to_csv('./my-submission.csv', index=False)